In [ ]:
from selenium import webdriver
import time
import pandas as pd
import requests
import sys
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from datetime import datetime,timedelta
import time
import random
import re
import operator

In [ ]:
from webdriver_manager.chrome import ChromeDriverManager

browser = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/raul.castillo/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache


In [ ]:
browser.get('https://www.tripadvisor.com.mx/Hotel_Review-g150797-d169771-Reviews-Camino_Real_Sumiya-Cuernavaca_Central_Mexico_and_Gulf_Coast.html#REVIEWS')

In [ ]:
data = pd.DataFrame(columns=('id','fecha_estancia','usuario','titulo','opinion','calificacion','genero'))

In [ ]:
elem = browser.find_element_by_xpath("//*")
soup = BeautifulSoup(elem.get_attribute('innerHTML'),'html.parser')

In [ ]:
rev = soup.find_all('q',"XllAv H4 _a")[0].get_text()

In [ ]:
len(rev)

215

In [ ]:
total = range(0,10,5) # En un rango de 0 a 10 reseñas, va a saltar de 5 en 5

In [ ]:
def clean_text(txt):
    txt = re.sub("[^a-záéíóúñüäë]", " ", txt.lower())
    # Nos pasa el textoo a minúsculas.
    txt = re.sub(' +',' ', txt)
    # Limpia nuestro texto.
    return txt.strip().split()
    # Nos regresa el texto con las delimitaciones especificadas.

def df_to_dict(df, key_column, val_column):
    """convierte dos pandas series en un diccionario"""
    xkey = df[key_column].tolist()
    xval = df[val_column].tolist()
    return dict(zip(xkey,xval))

def get_gender2(names):
    names = clean_text(names)
    names = [x for x in names if gender_list.get(x,'a') != 'a']
    gender ={'m':0, 'f':0, 'a':0}
    for i, name in enumerate(names):
        g = gender_list.get(name,'a')
        gender[g] += 1
        gender[g] += 2 if len(names) > 1 and i == 0 and g != 'a' else 0 
    gender['a'] = 0 if (gender['f']+gender['m']) > 0 else 1
    return max(gender.items(), key=operator.itemgetter(1))[0]

gender_list = pd.read_csv('nombres.csv')
gender_list = df_to_dict(gender_list, key_column='nombre', val_column='genero')

In [ ]:
url1 = "https://www.tripadvisor.com.mx/Hotel_Review-g2550249-d169771-Reviews"
url2 = "-Camino_Real_Sumiya_Cuernavaca-Jiutepec_Central_Mexico_and_Gulf_Coast.html"

In [ ]:
browser.maximize_window()
for p in total:
    if p == 0:
        centro = '-'
    else:
        centro = '-or'+str(p)+'-'
    url = url1+centro+url2
    print(url1+centro+url2)
    #print(url)
    browser.get(url)
    
    desplazar = random.randint(400, 1200)
    browser.execute_script('window.scrollTo(1, '+str(desplazar)+');')
    time.sleep(2)
    
    elem = browser.find_element_by_xpath("//*")
    soup = BeautifulSoup(elem.get_attribute('innerHTML'),'html.parser')
    samples = soup.find_all("div", "cWwQK MC R2 Gi z Z BB dXjiy")
    c=0
    for s in samples:
        ids = p+c+1
        sample = samples[c]
        autor = sample.find_all("a",'ui_header_link bPvDb')[0].get_text()
        gen = get_gender2(autor)
        cal = str(sample.find_all('div',"emWez F1")[0])
        cal = cal[92:len(cal)-15]
        titulo = sample.find_all('a',"fCitC")[0].get_text()
        opinion = sample.find_all('q',"XllAv H4 _a")[0].get_text()
        fecha_est = sample.find_all('span',"euPKI _R Me S4 H3")[0].get_text()
        fecha_est = fecha_est[21:]
        #data = data.append({'fecha_estancia': fecha_est,'usuario':nombre,'calificacion':cal,'titulo':titulo,'opinion':opinion},ignore_index=True)
        data = data.append({'id':ids, 'fecha_estancia': fecha_est,'usuario':autor,'calificacion':cal,'titulo':titulo,'opinion':opinion,'genero':gen},ignore_index=True)
        c = c+1
    
    p+5
    sleep = random.randint(5, 15)
    print(sleep)
    time.sleep(sleep)
          
print('Ya puedes exportar')

https://www.tripadvisor.com.mx/Hotel_Review-g2550249-d169771-Reviews--Camino_Real_Sumiya_Cuernavaca-Jiutepec_Central_Mexico_and_Gulf_Coast.html
14
https://www.tripadvisor.com.mx/Hotel_Review-g2550249-d169771-Reviews-or5--Camino_Real_Sumiya_Cuernavaca-Jiutepec_Central_Mexico_and_Gulf_Coast.html
8
Ya puedes exportar


In [ ]:
data

,id,fecha_estancia,usuario,titulo,opinion,calificacion,genero
0,1,mayo de 2022,vmgjmq,Padrisimo!!!m,"Muy bonito hotel, muy limpio, la comida muy ri...",40,a
1,2,marzo de 2022,lulumedinav,Mi estancia en el hotel fue buena,La habitación es agradable pero tuvimos la mal...,40,a
2,3,abril de 2022,378marianae,Excelente experiencia,La experiencia en el hotel fue gratificante y ...,50,a
3,4,abril de 2022,maria olivares,pesimas areas comunes,"no habia camastros, y se les hizo facil decir:...",20,f
4,5,abril de 2022,nriver577345,pésima atención,Tengo un problema con el cobro en recepción. M...,10,a
5,6,abril de 2022,carlosbrimor,Excelente servicio,Todo el personal tiene una gran actitud de ser...,40,a
6,7,abril de 2022,mtello379232,Esperaba mas al ser de Camino Real,"La propiedad es hermosa, buenísima ubicación, ...",30,a
7,8,abril de 2022,milkg297,Puede mejorar,Lugar bonito pero el servicio e instalaciones ...,30,a
8,9,abril de 2022,carmencardenas2022,Excelente servicio,Recomiendo que observen esta mejora. Si son 3...,50,a
9,10,abril de 2022,Rodrigo B,Mala relación calidad y precio,"Las instalaciones bien en general, las habitac...",30,m


In [ ]:
data.groupby(['calificacion']).count()

,id,fecha_estancia,usuario,titulo,opinion,genero
calificacion,,,,,,
10,1,1,1,1,1,1
20,1,1,1,1,1,1
30,3,3,3,3,3,3
40,3,3,3,3,3,3
50,2,2,2,2,2,2


In [ ]:
c=0
samples = soup.find_all("div", "cWwQK MC R2 Gi z Z BB dXjiy")
sample = samples[c]

In [ ]:
samples

[]

In [ ]:
nombre = sample.find_all("a",'ui_header_link bPvDb')[0].get_text()

In [ ]:
cal = str(sample.find_all('div',"emWez F1")[0])
cal = cal[92:len(cal)-15]

In [ ]:
titulo = sample.find_all('a',"fCitC")[0].get_text()

In [ ]:
opinion = sample.find_all('q',"XllAv H4 _a")[0].get_text()

In [ ]:
fecha_est = sample.find_all('span',"euPKI _R Me S4 H3")[0].get_text()
fecha_est = fecha_est[21:]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a4b48cf0-c50a-44f8-990b-e0da28b0964f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>